In [ ]:
import torch
import torchvision
from tqdm import tqdm
from torch.utils.data import DataLoader

In [ ]:
VGG_INPUT_SIZE = 224

normalize_transform = torchvision.transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((VGG_INPUT_SIZE, VGG_INPUT_SIZE)),
    torchvision.transforms.ToTensor(),
    normalize_transform
])

test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((VGG_INPUT_SIZE, VGG_INPUT_SIZE)),
    torchvision.transforms.ToTensor(),
    normalize_transform
])

train_dataset = torchvision.datasets.Food101(root="./data", download=True, split="train", transform=train_transform)
test_dataset = torchvision.datasets.Food101(root="./data", download=True, split="test", transform=test_transform)

In [ ]:
model = torchvision.models.vgg16_bn(weights=torchvision.models.VGG16_BN_Weights.DEFAULT)

num_classes = 101
in_features = model.classifier[-1].in_features

model.classifier[-1] = torch.nn.Linear(in_features=in_features, out_features=num_classes)

In [ ]:
BATCH_SIZE = 64
train_data_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_data_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE)

criterion = torch.nn.CrossEntropyLoss()
epochs_num = 10
total_step = len(train_data_loader)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Running on: {device}")

model = model.to(device=device)

learning_rate = 1e-5
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.005)


# print(f"training for {epochs_num} epochs with lr: {learning_rate}")

Running on: cpu
training for 10 epochs with lr: 1e-05


In [ ]:
for epoch in range(epochs_num):
    model.train()
    for i, (images, labels) in enumerate(tqdm(train_data_loader)):
        images = images.to(device=device)
        labels = labels.to(device=device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"\nEpoch [{epoch+1}/{epochs_num}], Step [{i+1}/{total_step}], Training Loss: {loss.item():.4f}")

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in tqdm(test_data_loader, desc="Validating"):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

        accuracy = 100 * correct / total
        print(f"Accuracy of the validation images: {accuracy:.2f}%")

    print(120 * '-')

In [ ]:
MODEL_PATH = 'vgg16_food101_finetuned.pth'

torch.save(model.state_dict(), MODEL_PATH)
print(f"saved to {MODEL_PATH}")